In [1]:
import cv2
import numpy as np

In [2]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,240)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,240)
while cap.isOpened():
    r,frame = cap.read()
    if r:
        frame = cv2.flip(frame,1)
        cv2.imshow("Camera",frame)
        if cv2.waitKey(1) & 0xff == ord("p"):
            cv2.imwrite("backGround.png",frame)
            break
    else: break
cap.release()
cv2.destroyAllWindows()

In [3]:
def empty(a):
    pass
    
cv2.namedWindow("Controls")
cv2.createTrackbar("hl","Controls",0,179,empty)
cv2.createTrackbar("sl","Controls",0,255,empty)
cv2.createTrackbar("vl","Controls",0,255,empty)
cv2.createTrackbar("hu","Controls",179,179,empty)
cv2.createTrackbar("su","Controls",255,255,empty)
cv2.createTrackbar("vu","Controls",255,255,empty)

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,240)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,240)
while cap.isOpened():
    r,frame = cap.read()
    if r:
        hl = cv2.getTrackbarPos("hl","Controls")
        sl = cv2.getTrackbarPos("sl","Controls")
        vl = cv2.getTrackbarPos("vl","Controls")
        hu = cv2.getTrackbarPos("hu","Controls")
        su = cv2.getTrackbarPos("su","Controls")
        vu = cv2.getTrackbarPos("vu","Controls")
        
        frame = cv2.flip(frame,1)
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        lower = np.array([hl,sl,vl]) 
        upper = np.array([hu,su,vu])
        mask = cv2.inRange(hsv,lower,upper)
        res = cv2.bitwise_and(frame,frame,mask=mask)
    
        cv2.imshow("Camera",frame)
        cv2.imshow("HSV",hsv)
        cv2.imshow("Mask",mask)
        cv2.imshow("Result",res)
        if cv2.waitKey(1) & 0xff == ord("p"):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

In [4]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,240)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,240)

# Load background image (make sure size matches camera frame)
back_img = cv2.imread("BackGround.png")
if back_img is None:
    print("❌ Error: could not load background image")
    exit()
back_img = cv2.resize(back_img, (240,240))

while cap.isOpened():
    r,frame = cap.read()
    if not r:
        break

    frame = cv2.flip(frame,1)
    frame = cv2.resize(frame,(240,240))
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)

    # Pick color range of cloth (currently set for white/gray range)
    lower = np.array([51,21,148])
    upper = np.array([116,255,255])
    mask = cv2.inRange(hsv,lower,upper)

    # Remove noise
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3),np.uint8))
    mask = cv2.dilate(mask, np.ones((3,3), np.uint8))

    # Inverse mask to get rest of the scene
    mask_inv = cv2.bitwise_not(mask)

    # Take only non-cloth part from current frame
    part1 = cv2.bitwise_and(frame, frame, mask=mask_inv)

    # Take cloak area from background
    part2 = cv2.bitwise_and(back_img, back_img, mask=mask)

    # Combine both
    res = cv2.addWeighted(part1, 1, part2, 1, 0)

    cv2.imshow("Camera",frame)
    cv2.imshow("Mask",mask)
    cv2.imshow("part1",part1)
    cv2.imshow("part2",part2)
    cv2.imshow("Result",res)

    if cv2.waitKey(1) & 0xff == ord("p"):
        break

cap.release()
cv2.destroyAllWindows()
